In [1]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.1.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:2 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Reading package lists... Done


In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-11-05 02:10:31--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar.3’

postgresql-42.2.9.j 100%[===================>] 892.61K  --.-KB/s    in 0.1s    

2022-11-05 02:10:32 (5.92 MB/s) - ‘postgresql-42.2.9.jar.3’ saved [914037/914037]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("PlaysETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [4]:
# Read in data from GCS Buckets
from pyspark import SparkFiles
url="https://storage.googleapis.com/big-data-bowl/pffScoutingData.csv"
spark.sparkContext.addFile(url)
pffScouting_data_df = spark.read.csv(SparkFiles.get("pffScoutingData.csv"), sep=",", header=True, inferSchema=True)

# Show DataFrame
pffScouting_data_df.show()

+----------+------+-----+----------+-------------------+-------+---------+--------+--------------------+--------------+----------------+---------------+----------------------+-------------+------------------+
|    gameId|playId|nflId|  pff_role|pff_positionLinedUp|pff_hit|pff_hurry|pff_sack|pff_beatenByDefender|pff_hitAllowed|pff_hurryAllowed|pff_sackAllowed|pff_nflIdBlockedPlayer|pff_blockType|pff_backFieldBlock|
+----------+------+-----+----------+-------------------+-------+---------+--------+--------------------+--------------+----------------+---------------+----------------------+-------------+------------------+
|2021090900|    97|25511|      Pass|                 QB|     NA|       NA|      NA|                  NA|            NA|              NA|             NA|                    NA|           NA|                NA|
|2021090900|    97|35481|Pass Route|               TE-L|     NA|       NA|      NA|                  NA|            NA|              NA|             NA|            

In [6]:
from pyspark.sql.types import IntegerType
pffScouting_data_df = pffScouting_data_df.withColumn("pff_hit", pffScouting_data_df["pff_hit"].cast(IntegerType()))
pffScouting_data_df = pffScouting_data_df.withColumn("pff_hurry", pffScouting_data_df["pff_hurry"].cast(IntegerType()))
pffScouting_data_df = pffScouting_data_df.withColumn("pff_sack", pffScouting_data_df["pff_sack"].cast(IntegerType()))
pffScouting_data_df = pffScouting_data_df.withColumn("pff_beatenByDefender", pffScouting_data_df["pff_beatenByDefender"].cast(IntegerType()))
pffScouting_data_df = pffScouting_data_df.withColumn("pff_hitAllowed", pffScouting_data_df["pff_hitAllowed"].cast(IntegerType()))
pffScouting_data_df = pffScouting_data_df.withColumn("pff_hurryAllowed", pffScouting_data_df["pff_hurryAllowed"].cast(IntegerType()))
pffScouting_data_df = pffScouting_data_df.withColumn("pff_sackAllowed", pffScouting_data_df["pff_sackAllowed"].cast(IntegerType()))
pffScouting_data_df = pffScouting_data_df.withColumn("pff_nflIdBlockedPlayer", pffScouting_data_df["pff_nflIdBlockedPlayer"].cast(IntegerType()))
pffScouting_data_df = pffScouting_data_df.withColumn("pff_backFieldBlock", pffScouting_data_df["pff_backFieldBlock"].cast(IntegerType()))

from pyspark.sql.functions import *
pffScouting_data_df = pffScouting_data_df.withColumn('pff_blockType', regexp_replace('pff_blockType', 'NA', ''))

pffScouting_data_df.show()

+----------+------+-----+----------+-------------------+-------+---------+--------+--------------------+--------------+----------------+---------------+----------------------+-------------+------------------+
|    gameId|playId|nflId|  pff_role|pff_positionLinedUp|pff_hit|pff_hurry|pff_sack|pff_beatenByDefender|pff_hitAllowed|pff_hurryAllowed|pff_sackAllowed|pff_nflIdBlockedPlayer|pff_blockType|pff_backFieldBlock|
+----------+------+-----+----------+-------------------+-------+---------+--------+--------------------+--------------+----------------+---------------+----------------------+-------------+------------------+
|2021090900|    97|25511|      Pass|                 QB|   null|     null|    null|                null|          null|            null|           null|                  null|             |              null|
|2021090900|    97|35481|Pass Route|               TE-L|   null|     null|    null|                null|          null|            null|           null|            

Postgres Setup

In [7]:
# Store environment variable
from getpass import getpass
password = getpass('Enter database password')

# Configure settings for Cloud SQL
mode = "append"
jdbc_url="jdbc:postgresql://34.72.136.99:5432/big-data-bowl"
config = {"user":"postgres", 
          "password": password, 
          "driver":"org.postgresql.Driver"}


Enter database password··········


In [10]:
# Write DataFrame to pffscouting table in Cloud SQL
pffScouting_data_df.write.jdbc(url=jdbc_url, table='pffscoutingdata', mode=mode, properties=config)